# Milestone 2

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# all imports
from src import *
import pandas as pd
import os.path

## Data acquisition

In [ ]:
filename = './data/dataframe_2016_to_2019.csv'
start_year = 2016
end_year = 2019

if not os.path.isfile(filename):
    # Get the data from the NHL API (2016 - 2019)
    nhl_data_provider = get_data_from(start_year, end_year)

    # Clean the data
    clean_regular_season, clean_playoff = clean_data(nhl_data_provider)

    # Transform data into dataframe
    df_2016_to_2019 = convert_dictionaries_to_dataframes(clean_regular_season,
                                                     clean_playoff,
                                                     np.arange(start_year, end_year + 1).tolist())
    df_2016_to_2019.to_csv(filename, index=False)

Successfully imported NHL data for the 2016 season.
Successfully imported NHL data for the 2017 season.
Successfully imported NHL data for the 2018 season.
Successfully imported NHL data for the 2019 season.


In [ ]:
train_data = pd.read_csv(filename)

# Ingénierie des caractéristiques

## Question 1

## Question 2

## Question 3

In [ ]:
plot_goals_by_distance(train_data)